In [20]:
from textprocessor import CustomTextProcessorNoStem
from textprocessor import CustomTextProcessor

import xml.etree.ElementTree as ET
from tqdm import tqdm
import zipfile
import re
import os

In [21]:
processor = CustomTextProcessor()

collection_name = "XML-Coll-withSem"
processed_dir = "../lib/processed_data/" + collection_name + "_" + processor.get_text_processor_name() + "/"

os.makedirs(processed_dir, exist_ok=True)

In [22]:
zip_file_path = "../lib/data/practice_05/" + collection_name + ".zip"

In [23]:
def process_text_in_tags(element):
    if element.text:
        processed_text = ' '.join(processor.pre_processing(element.text))
        element.text = processed_text

    if element.tail:  # Process the tail (text after the closing tag)
        processed_tail = ' '.join(processor.pre_processing(element.tail))
        element.tail = processed_tail

    for child in element:
        process_text_in_tags(child)

In [24]:
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    xml_file_name = zip_ref.namelist()
    for file in tqdm(xml_file_name, desc="Processing files"):
        with zip_ref.open(file) as xml_file:
            xml_content = xml_file.read().decode('utf-8')

            root = ET.fromstring(re.sub('&[^;]+;', ' ', xml_content))
            process_text_in_tags(root)

            # Convert the processed XML content to a string
            processed_xml = ET.tostring(root, encoding='unicode')
            processed_xml_file_path = f"{processed_dir}{file.split('/')[1].split('.')[0]}.xml"
            
            # Save the processed XML content to a new XML file
            with open(processed_xml_file_path, 'w', encoding='utf-8') as processed_file:
                processed_file.write(processed_xml)

Processing files: 100%|██████████| 9804/9804 [11:18<00:00, 14.46it/s]
